# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb170c03f10>
├── 'a' --> tensor([[ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163]])
└── 'x' --> <FastTreeValue 0x7fb170c03f70>
    └── 'c' --> tensor([[-0.2069,  1.3491, -0.5355, -0.7272],
                        [-1.0384,  1.0513, -1.4774, -0.3798],
                        [-1.1206, -0.6122,  2.2145, -0.3983]])

In [4]:
t.a

tensor([[ 0.2381,  0.9779, -1.0119],
        [ 1.0029, -0.4537,  0.0163]])

In [5]:
%timeit t.a

61.2 ns ± 0.304 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb170c03f10>
├── 'a' --> tensor([[ 0.6703, -1.2904,  0.1642],
│                   [-0.4736,  0.6032,  0.9351]])
└── 'x' --> <FastTreeValue 0x7fb170c03f70>
    └── 'c' --> tensor([[-0.2069,  1.3491, -0.5355, -0.7272],
                        [-1.0384,  1.0513, -1.4774, -0.3798],
                        [-1.1206, -0.6122,  2.2145, -0.3983]])

In [7]:
%timeit t.a = new_value

62.6 ns ± 0.0247 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163]]),
    x: Batch(
           c: tensor([[-0.2069,  1.3491, -0.5355, -0.7272],
                      [-1.0384,  1.0513, -1.4774, -0.3798],
                      [-1.1206, -0.6122,  2.2145, -0.3983]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2381,  0.9779, -1.0119],
        [ 1.0029, -0.4537,  0.0163]])

In [11]:
%timeit b.a

56.8 ns ± 0.0254 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3955,  0.2643, -0.7864],
               [ 0.8623, -0.1680, -1.4082]]),
    x: Batch(
           c: tensor([[-0.2069,  1.3491, -0.5355, -0.7272],
                      [-1.0384,  1.0513, -1.4774, -0.3798],
                      [-1.1206, -0.6122,  2.2145, -0.3983]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.0783 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 0.292 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 654 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb0c58b1760>
├── 'a' --> tensor([[[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]],
│           
│                   [[ 0.2381,  0.9779, -1.0119],
│                    [ 1.0029, -0.4537,  0.0163]]])
└── 'x' --> <FastTreeValue 0x7fb0c58b1d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 55.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb0c592dbb0>
├── 'a' --> tensor([[ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163],
│                   [ 0.2381,  0.9779, -1.0119],
│                   [ 1.0029, -0.4537,  0.0163]])
└── 'x' --> <FastTreeValue 0x7fb0c5eb1b50>
    └── 'c' --> tensor([[-0.2069,  1.3491, -0.5355, -0.7272],
                        [-1.0384,  1.0513, -1.4774, -0.3798],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 80.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]],
       
               [[ 0.2381,  0.9779, -1.0119],
                [ 1.0029, -0.4537,  0.0163]]]),
    x: Batch(
           c: tensor([[[-0.2069,  1.3491, -0.5355, -0.7272],
                       [-1.0384,  1.0513, -1.4774, -0.3798],
                       [-1.1206, -0.6122,  2.2145, -0.3983]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 268 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163],
               [ 0.2381,  0.9779, -1.0119],
               [ 1.0029, -0.4537,  0.0163]]),
    x: Batch(
           c: tensor([[-0.2069,  1.3491, -0.5355, -0.7272],
                      [-1.0384,  1.0513, -1.4774, -0.3798],
                      [-1.1206, -0.6122,  2.2145, -0.3983],
                      [-0.2069,  1.3491, -0.5355, -0.7272],
                      [-1.0384,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 253 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 5.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
